In [1]:
import usage
# Set the PAT 
usage.PAT = open("/Users/aniket/ws/tokens/dev.token").read()

In order to look at, say a month worth of, daily usage per project with a split by engines within each project, there has to be two steps
1. Capture the list of projects in the Org and corresponding engines
  - The [projects](https://docs.dremio.com/cloud/reference/api/projects#listing-all-projects) API gives the list of projects and its attributes
  - For each of them, the [engines](https://docs.dremio.com/cloud/reference/api/engines#listing-all-engines) API gives the corresponding projects
2. Capture the daily usage per project and then subdivide the usage, by day, into its engines for the projects 
  - The [usage](https://docs.dremio.com/cloud/reference/api/usage) API with `frequency=DAILY` will list daily usage per project
  - Per day, per project the same API can give engine level details with `groupBy=ENGINE` and `filter==project_id==.. && start_time >= ..`

Finally steps 1 & 2 can be _joined_ to yield a full view
 

In [2]:
pe_df = usage.build_projects_and_engines_table()
pe_df.head()

,project_id,project_name,engine_id,engine_name,engine_size,instanceFamily
0,8c8a837b-5cda-4e75-8684-17a83f0e0b24,n,3283a27e-5145-44ea-91a7-0c1c4e3ef29c,preview,XX_SMALL_V1,
1,26fcc0d9-b9d5-4c7f-b935-30b4e4a0ebab,shaowen,cc6ec3ef-36e7-4028-b2aa-66d7098eaa70,preview,XX_SMALL_V1,M5D
2,ba164cfc-e77c-4753-895c-a8a92576964e,test_default_instance_family,069acb17-2bad-4533-9f87-f5ba7d964537,f04b1,XX_SMALL_V1,M6ID
3,ba164cfc-e77c-4753-895c-a8a92576964e,test_default_instance_family,817f6177-63ab-40a1-a801-9579f93e93c2,preview,XX_SMALL_V1,M5D
4,ba164cfc-e77c-4753-895c-a8a92576964e,test_default_instance_family,e08dfcda-8085-46da-a9b5-ea52d8667956,test,XX_SMALL_V1,M5D


In [3]:
pe_usage_df = usage.build_usage_per_engine_table()
pe_usage_df.head()

,project_id,start_time,end_time,project_usage,engine_id,engine_start,engine_usage
0,054d40f2-2338-440a-97aa-97933ab207a4,2024-05-30 00:00:00+00:00,2024-05-31 00:00:00+00:00,4.000,2cf0a6fb-1e2d-4ef9-b065-af909d99335e,2024-05-30 00:00:00+00:00,4.000
1,054d40f2-2338-440a-97aa-97933ab207a4,2024-05-29 00:00:00+00:00,2024-05-30 00:00:00+00:00,3.781,2cf0a6fb-1e2d-4ef9-b065-af909d99335e,2024-05-29 00:00:00+00:00,3.781
2,054d40f2-2338-440a-97aa-97933ab207a4,2024-05-28 00:00:00+00:00,2024-05-29 00:00:00+00:00,4.003,2cf0a6fb-1e2d-4ef9-b065-af909d99335e,2024-05-28 00:00:00+00:00,4.003
3,054d40f2-2338-440a-97aa-97933ab207a4,2024-05-23 00:00:00+00:00,2024-05-24 00:00:00+00:00,4.012,2cf0a6fb-1e2d-4ef9-b065-af909d99335e,2024-05-23 00:00:00+00:00,4.012
4,054d40f2-2338-440a-97aa-97933ab207a4,2024-05-22 00:00:00+00:00,2024-05-23 00:00:00+00:00,4.008,2cf0a6fb-1e2d-4ef9-b065-af909d99335e,2024-05-22 00:00:00+00:00,4.008


In [4]:
full_usage = pe_df.join(pe_usage_df.set_index(['project_id', 'engine_id']), on=[
           'project_id', 'engine_id'], how='inner')
full_usage.head()

,project_id,project_name,engine_id,engine_name,engine_size,instanceFamily,start_time,end_time,project_usage,engine_start,engine_usage
0,8c8a837b-5cda-4e75-8684-17a83f0e0b24,n,3283a27e-5145-44ea-91a7-0c1c4e3ef29c,preview,XX_SMALL_V1,,2024-05-08 00:00:00+00:00,2024-05-09 00:00:00+00:00,0.074,2024-05-08 00:00:00+00:00,0.074
1,26fcc0d9-b9d5-4c7f-b935-30b4e4a0ebab,shaowen,cc6ec3ef-36e7-4028-b2aa-66d7098eaa70,preview,XX_SMALL_V1,M5D,2024-05-23 00:00:00+00:00,2024-05-24 00:00:00+00:00,4.105,2024-05-23 00:00:00+00:00,4.105
1,26fcc0d9-b9d5-4c7f-b935-30b4e4a0ebab,shaowen,cc6ec3ef-36e7-4028-b2aa-66d7098eaa70,preview,XX_SMALL_V1,M5D,2024-05-21 00:00:00+00:00,2024-05-22 00:00:00+00:00,7.432,2024-05-21 00:00:00+00:00,7.432
1,26fcc0d9-b9d5-4c7f-b935-30b4e4a0ebab,shaowen,cc6ec3ef-36e7-4028-b2aa-66d7098eaa70,preview,XX_SMALL_V1,M5D,2024-05-09 00:00:00+00:00,2024-05-10 00:00:00+00:00,65.783,2024-05-09 00:00:00+00:00,65.783
1,26fcc0d9-b9d5-4c7f-b935-30b4e4a0ebab,shaowen,cc6ec3ef-36e7-4028-b2aa-66d7098eaa70,preview,XX_SMALL_V1,M5D,2024-05-08 00:00:00+00:00,2024-05-10 00:00:00+00:00,6.518,2024-05-09 00:00:00+00:00,4.000


In [6]:
full_usage[["start_time", "project_name", "engine_size", "engine_usage"]].groupby(
    by=["start_time", "project_name", "engine_size"]).sum()

engine_usage
start_time                project_name               engine_size              
2024-05-01 00:00:00+00:00 dogfood-aws                X4_SMALL_V1      4397.298
                                                     XX_SMALL_V1       547.365
                          ingestion_test_project     XX_SMALL_V1       112.487
                          sheev_catalog              XX_SMALL_V1        11.964
                          wonjae-test                XX_SMALL_V1         4.021
...                                                                        ...
2024-05-30 00:00:00+00:00 yuzong_test_project        XX_SMALL_V1         4.000
2024-05-31 00:00:00+00:00 azure_test_3               XX_SMALL_V1         4.212
                          dogfood-aws                X4_SMALL_V1        12.132
                          ingestion_e2e_test_project XX_SMALL_V1         8.043
                          ingestion_test_project     XX_SMALL_V1         4.017

[123 rows x 1 columns]